# Análisis Exploratorio: Migraciones de Investigadores Científicos 🔬🌍

## Contexto

Este notebook analiza los patrones de migración de investigadores científicos a nivel global, identificando:

- **Países atractores vs. emisores** de talento científico
- **Corredores migratorios principales** (rutas más frecuentes)
- **Saldo neto de brain gain/brain drain** por país
- **Relación con desarrollo socioeconómico** (PIB per cápita, gasto I+D)

## Datasets

- `migration_flows.parquet|csv`: Flujos agregados origen→destino
- `migrations_clean.parquet|csv`: Registros individuales de investigadores
- `wdi_indicators.parquet|csv`: Indicadores del Banco Mundial (población, PIB, I+D)
- `country_mapping.csv`: Mapeo ISO2→ISO3

---

## 1. Setup: Importación de Librerías y Configuración

In [8]:
# Librerías estándar
import pandas as pd
import numpy as np
from pathlib import Path
import warnings

# Visualización
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# Tema de plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

# Paleta de colores personalizada
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'accent': '#F18F01',
    'success': '#06A77D',
    'warning': '#D00000',
    'neutral': '#6C757D'
}

print("✓ Librerías importadas correctamente")
print(f"  - pandas: {pd.__version__}")
print(f"  - plotly: {plotly.__version__}")

✓ Librerías importadas correctamente
  - pandas: 2.3.3
  - plotly: 6.3.1


## 2. Carga de Datos Procesados

Cargamos los datasets generados en el notebook de preprocesamiento (`prep.ipynb`).

In [9]:
# Rutas
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / 'outputs' / 'processed'

print(f"📁 Directorio de datos: {DATA_DIR}\n")

# Función para carga dual (Parquet → CSV)
def load_dataset(base_name, description):
    """Carga Parquet si existe, sino CSV."""
    parquet_path = DATA_DIR / f"{base_name}.parquet"
    csv_path = DATA_DIR / f"{base_name}.csv"
    
    if parquet_path.exists():
        df = pd.read_parquet(parquet_path)
        print(f"✓ {description}: {len(df):,} registros (Parquet)")
    elif csv_path.exists():
        df = pd.read_csv(csv_path)
        print(f"✓ {description}: {len(df):,} registros (CSV)")
    else:
        print(f"⚠️  {description}: No encontrado")
        return None
    
    return df

# Cargar datasets
df_flows = load_dataset('migration_flows', 'Flujos migratorios')
df_migrations = load_dataset('migrations_clean', 'Migraciones individuales')
df_wdi = load_dataset('wdi_indicators', 'Indicadores WDI')
df_mapping = pd.read_csv(DATA_DIR / 'country_mapping.csv') if (DATA_DIR / 'country_mapping.csv').exists() else None

print(f"\n✓ Datasets cargados exitosamente")

📁 Directorio de datos: c:\Users\José Luis\Documents\GitHub\Scientific-Researcher-Migrations\outputs\processed

✓ Flujos migratorios: 4,249 registros (Parquet)
✓ Migraciones individuales: 604,173 registros (Parquet)
✓ Indicadores WDI: 26,942 registros (Parquet)

✓ Datasets cargados exitosamente
✓ Migraciones individuales: 604,173 registros (Parquet)
✓ Indicadores WDI: 26,942 registros (Parquet)

✓ Datasets cargados exitosamente


## 3. Exploración Inicial de Flujos Migratorios

In [10]:
print("📊 RESUMEN DE FLUJOS MIGRATORIOS\n" + "="*70)

print(f"\n📈 Estadísticas Generales:")
print(f"  - Total de rutas migratorias: {len(df_flows):,}")
print(f"  - Total de investigadores que migraron: {df_flows['n_researchers'].sum():,}")
print(f"  - Países origen únicos: {df_flows['origin'].nunique()}")
print(f"  - Países destino únicos: {df_flows['destination'].nunique()}")
print(f"  - Media de investigadores por ruta: {df_flows['n_researchers'].mean():.1f}")
print(f"  - Mediana de investigadores por ruta: {df_flows['n_researchers'].median():.0f}")

print(f"\n🔝 Top 20 Rutas Migratorias:\n")
display(df_flows[[
    'route', 'origin', 'destination', 'n_researchers', 
    'phd_year_mean', 'origin_year_mean'
]].head(20))

📊 RESUMEN DE FLUJOS MIGRATORIOS

📈 Estadísticas Generales:
  - Total de rutas migratorias: 4,249
  - Total de investigadores que migraron: 62,004
  - Países origen únicos: 194
  - Países destino únicos: 202
  - Media de investigadores por ruta: 14.6
  - Mediana de investigadores por ruta: 2

🔝 Top 20 Rutas Migratorias:



,route,origin,destination,n_researchers,phd_year_mean,origin_year_mean
0,CN → US,CN,US,3508,2011,2000
1,IN → US,IN,US,1916,2009,1998
2,CA → US,CA,US,1029,2004,1995
3,GB → US,GB,US,940,2000,1992
4,US → GB,US,GB,786,2006,1997
5,GB → AU,GB,AU,719,2000,1991
6,US → KR,US,KR,709,2000,1995
7,US → CA,US,CA,616,2001,1993
8,US → TW,US,TW,610,1997,1992
9,KR → US,KR,US,562,2010,1997


## 4. Análisis de Países: Top Emisores y Receptores

### 4.1 Top Países Emisores (Brain Drain)

In [11]:
# Agregación por país origen (emisores)
top_emitters = df_flows.groupby('origin')['n_researchers'].sum().sort_values(ascending=False).reset_index()
top_emitters.columns = ['country', 'total_emigrants']
top_emitters['rank'] = range(1, len(top_emitters) + 1)

print(f"🔴 TOP 15 PAÍSES EMISORES DE CIENTÍFICOS (Brain Drain)\n" + "="*70)
display(top_emitters.head(15))

# Visualización
fig = px.bar(
    top_emitters.head(15),
    x='total_emigrants',
    y='country',
    orientation='h',
    title='Top 15 Países Emisores de Investigadores Científicos',
    labels={'total_emigrants': 'Investigadores Emigrados', 'country': 'País Origen'},
    color='total_emigrants',
    color_continuous_scale='Reds',
    text='total_emigrants'
)
fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig.update_layout(height=500, showlegend=False, yaxis={'categoryorder':'total ascending'})
fig.show()

🔴 TOP 15 PAÍSES EMISORES DE CIENTÍFICOS (Brain Drain)


,country,total_emigrants,rank
0,US,7657,1
1,CN,6187,2
2,GB,5376,3
3,IN,4559,4
4,DE,2838,5
5,IT,2351,6
6,FR,2280,7
7,CA,2275,8
8,ES,1921,9
9,RU,1417,10


### 4.2 Top Países Receptores (Brain Gain)

In [12]:
# Agregación por país destino (receptores)
top_receivers = df_flows.groupby('destination')['n_researchers'].sum().sort_values(ascending=False).reset_index()
top_receivers.columns = ['country', 'total_immigrants']
top_receivers['rank'] = range(1, len(top_receivers) + 1)

print(f"🟢 TOP 15 PAÍSES RECEPTORES DE CIENTÍFICOS (Brain Gain)\n" + "="*70)
display(top_receivers.head(15))

# Visualización
fig = px.bar(
    top_receivers.head(15),
    x='total_immigrants',
    y='country',
    orientation='h',
    title='Top 15 Países Receptores de Investigadores Científicos',
    labels={'total_immigrants': 'Investigadores Recibidos', 'country': 'País Destino'},
    color='total_immigrants',
    color_continuous_scale='Greens',
    text='total_immigrants'
)
fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig.update_layout(height=500, showlegend=False, yaxis={'categoryorder':'total ascending'})
fig.show()

🟢 TOP 15 PAÍSES RECEPTORES DE CIENTÍFICOS (Brain Gain)


,country,total_immigrants,rank
0,US,14547,1
1,GB,6651,2
2,AU,3823,3
3,DE,2260,4
4,CA,2249,5
5,ES,1907,6
6,SE,1743,7
7,FR,1453,8
8,KR,1370,9
9,CH,1355,10


## 5. Saldo Migratorio Neto: Atractores vs. Exportadores

Calculamos el **saldo neto** = (Inmigración - Emigración) para identificar países atractores y exportadores de talento científico.

In [13]:
# Calcular inmigración y emigración por país
immigration = df_flows.groupby('destination')['n_researchers'].sum().reset_index()
immigration.columns = ['country', 'immigration']

emigration = df_flows.groupby('origin')['n_researchers'].sum().reset_index()
emigration.columns = ['country', 'emigration']

# Merge y calcular saldo neto
net_migration = immigration.merge(emigration, on='country', how='outer').fillna(0)
net_migration['net_balance'] = net_migration['immigration'] - net_migration['emigration']
net_migration['total_flow'] = net_migration['immigration'] + net_migration['emigration']
net_migration['migration_ratio'] = np.where(
    net_migration['emigration'] > 0,
    net_migration['immigration'] / net_migration['emigration'],
    np.inf
)

# Clasificar países
net_migration['type'] = np.where(
    net_migration['net_balance'] > 0, 'Atractor', 'Exportador'
)

net_migration = net_migration.sort_values('net_balance', ascending=False).reset_index(drop=True)

print(f"⚖️  SALDO MIGRATORIO NETO\n" + "="*70)
print(f"\nTop 10 Atractores (Brain Gain):")
display(net_migration[
    ['country', 'immigration', 'emigration', 'net_balance', 'migration_ratio']
].head(10))

print(f"\nTop 10 Exportadores (Brain Drain):")
display(net_migration[
    ['country', 'immigration', 'emigration', 'net_balance', 'migration_ratio']
].tail(10))

⚖️  SALDO MIGRATORIO NETO

Top 10 Atractores (Brain Gain):


,country,immigration,emigration,net_balance,migration_ratio
0,US,14547,7657,6890,1.90
1,AU,3823,1116,2707,3.43
2,GB,6651,5376,1275,1.24
3,SE,1743,514,1229,3.39
4,SA,1094,71,1023,15.41
5,DK,1145,283,862,4.05
6,MY,1074,269,805,3.99
7,CH,1355,574,781,2.36
8,QA,589,11,578,53.55
9,SG,774,244,530,3.17



Top 10 Exportadores (Brain Drain):


,country,immigration,emigration,net_balance,migration_ratio
219,BD,105,481,-376,0.22
220,PK,219,676,-457,0.32
221,DE,2260,2838,-578,0.80
222,GR,175,881,-706,0.20
223,FR,1453,2280,-827,0.64
224,IR,304,1213,-909,0.25
225,RU,330,1417,-1087,0.23
226,IT,956,2351,-1395,0.41
227,IN,373,4559,-4186,0.08
228,CN,1209,6187,-4978,0.20


In [14]:
# Visualización del saldo neto (top 20 atractores y exportadores)
top_attractors = net_migration.head(20).copy()
top_exporters = net_migration.tail(20).copy()
viz_data = pd.concat([top_attractors, top_exporters])

fig = px.bar(
    viz_data,
    x='net_balance',
    y='country',
    orientation='h',
    title='Saldo Migratorio Neto: Top 20 Atractores y Exportadores',
    labels={'net_balance': 'Saldo Neto (Inmigración - Emigración)', 'country': 'País'},
    color='net_balance',
    color_continuous_scale='RdYlGn',
    color_continuous_midpoint=0,
    text='net_balance'
)
fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig.update_layout(
    height=700,
    yaxis={'categoryorder':'total ascending'},
    showlegend=False
)
fig.add_vline(x=0, line_dash="dash", line_color="gray", annotation_text="Balance = 0")
fig.show()

## 6. Análisis de Corredores Migratorios

Identificamos los **corredores bilaterales** más importantes (rutas específicas origen→destino).

In [15]:
# Top 20 corredores
top_corridors = df_flows.nlargest(20, 'n_researchers')[[
    'route', 'origin', 'destination', 'n_researchers',
    'phd_year_mean', 'origin_year_mean'
]].copy()

print(f"🛤️  TOP 20 CORREDORES MIGRATORIOS\n" + "="*70)
display(top_corridors)

# Visualización
fig = px.bar(
    top_corridors,
    x='n_researchers',
    y='route',
    orientation='h',
    title='Top 20 Corredores Migratorios Más Importantes',
    labels={'n_researchers': 'Número de Investigadores', 'route': 'Corredor (Origen → Destino)'},
    color='n_researchers',
    color_continuous_scale='Blues',
    text='n_researchers'
)
fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig.update_layout(height=600, showlegend=False, yaxis={'categoryorder':'total ascending'})
fig.show()

🛤️  TOP 20 CORREDORES MIGRATORIOS


,route,origin,destination,n_researchers,phd_year_mean,origin_year_mean
0,CN → US,CN,US,3508,2011,2000
1,IN → US,IN,US,1916,2009,1998
2,CA → US,CA,US,1029,2004,1995
3,GB → US,GB,US,940,2000,1992
4,US → GB,US,GB,786,2006,1997
5,GB → AU,GB,AU,719,2000,1991
6,US → KR,US,KR,709,2000,1995
7,US → CA,US,CA,616,2001,1993
8,US → TW,US,TW,610,1997,1992
9,KR → US,KR,US,562,2010,1997


## 7. Diagrama de Flujos: Sankey Diagram

### Función para crear Sankey Diagram

**Limitaciones de Chord Diagram en Python:**
- No existe librería nativa robusta en Python (a diferencia de R con `circlize`)
- Librerías como `holoviews` o `chord` son poco mantenidas y difíciles de configurar

**Alternativa: Sankey Diagram con Plotly**

✅ **Pros:**
- Nativo en Plotly (sin dependencias adicionales)
- Interactivo (hover, zoom)
- Fácil de personalizar
- Muestra magnitud y dirección de flujos claramente
- Escalable a muchos nodos

⚠️ **Contras:**
- No muestra relaciones circulares como un chord diagram
- Layout automático puede solapar etiquetas con muchos nodos
- Menos compacto visualmente para muchas conexiones

In [16]:
def make_sankey(df_edges, top_n=20, title="Flujos Migratorios"):
    """
    Crea un Sankey Diagram interactivo a partir de flujos migratorios.
    
    Parameters:
    -----------
    df_edges : DataFrame
        DataFrame con columnas: 'origin_iso3', 'destination_iso3', 'n_researchers'
    top_n : int
        Número de flujos principales a visualizar (por defecto 20)
    title : str
        Título del diagrama
    
    Returns:
    --------
    plotly.graph_objects.Figure
    """
    # Filtrar top N flujos
    df_top = df_edges.nlargest(top_n, 'n_researchers').copy()
    
    # Crear mapeo de nodos únicos (origen + destino)
    all_countries = list(set(df_top['origin_iso3'].tolist() + df_top['destination_iso3'].tolist()))
    country_to_idx = {country: idx for idx, country in enumerate(all_countries)}
    
    # Crear listas para Sankey
    sources = [country_to_idx[origin] for origin in df_top['origin_iso3']]
    targets = [country_to_idx[dest] for dest in df_top['destination_iso3']]
    values = df_top['n_researchers'].tolist()
    
    # Colores por región (simplificado)
    node_colors = ['rgba(31, 119, 180, 0.8)'] * len(all_countries)
    link_colors = ['rgba(31, 119, 180, 0.3)'] * len(values)
    
    # Crear diagrama
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="white", width=0.5),
            label=all_countries,
            color=node_colors
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            color=link_colors
        )
    )])
    
    fig.update_layout(
        title=title,
        font=dict(size=12),
        height=700
    )
    
    return fig

print("✓ Función make_sankey() definida")
print("\nUso: fig = make_sankey(df_flows, top_n=30, title='Mi Título')")
print("     fig.show()")

✓ Función make_sankey() definida

Uso: fig = make_sankey(df_flows, top_n=30, title='Mi Título')
     fig.show()


In [17]:
# Aplicar función a los top 30 flujos
fig_sankey = make_sankey(
    df_flows,
    top_n=30,
    title="Top 30 Flujos Migratorios de Investigadores Científicos"
)
fig_sankey.show()

## 8. Integración con World Development Indicators (WDI)

### 8.1 Preparación de Datos WDI

In [18]:
if df_wdi is not None:
    print(f"🌐 ANÁLISIS CON INDICADORES WDI\n" + "="*70)
    
    # Filtrar año más reciente disponible (2015-2016 para alinear con dataset)
    wdi_recent = df_wdi[df_wdi['Year'].between(2014, 2016)].copy()
    
    # Pivot para tener indicadores como columnas
    wdi_pivot = wdi_recent.pivot_table(
        index='iso3',
        columns='IndicatorCode',
        values='Value',
        aggfunc='mean'  # Promedio si hay múltiples años
    ).reset_index()
    
    # Renombrar columnas
    wdi_pivot.columns = [
        'country',
        'gdp_per_capita',
        'rd_expenditure_pct',
        'population',
        'researchers_per_million'
    ]
    
    print(f"✓ Indicadores WDI procesados: {len(wdi_pivot)} países")
    print(f"\n📋 Vista previa:")
    display(wdi_pivot.head(10))
else:
    print("⚠️  Indicadores WDI no disponibles. Saltando análisis.")
    wdi_pivot = None

🌐 ANÁLISIS CON INDICADORES WDI
✓ Indicadores WDI procesados: 247 países

📋 Vista previa:


,country,gdp_per_capita,rd_expenditure_pct,population,researchers_per_million
0,ABW,NaN,NaN,NaN,"103,441.00"
1,ADO,NaN,NaN,NaN,"72,786.00"
2,AFG,NaN,633.57,NaN,"31,627,506.00"
3,AGO,NaN,NaN,NaN,"24,227,524.00"
4,ALB,NaN,"4,564.39",NaN,"2,894,475.00"
5,ARB,NaN,"7,386.43",NaN,"385,272,539.00"
6,ARE,NaN,"43,962.71",NaN,"9,086,139.00"
7,ARG,NaN,"12,509.53",NaN,"42,980,026.00"
8,ARM,NaN,"3,873.53",NaN,"3,006,154.00"
9,ASM,NaN,NaN,NaN,"55,434.00"


### 8.2 Tasas de Migración Per Cápita

In [19]:
if wdi_pivot is not None:
    # Merge con saldo migratorio
    # Primero necesitamos iso3 en net_migration
    if df_mapping is not None:
        net_migration_iso3 = net_migration.merge(
            df_mapping[['iso2', 'iso3']],
            left_on='country',
            right_on='iso2',
            how='left'
        )
    else:
        # Si no hay mapeo, asumir que country ya es iso2 y buscar en flows
        iso_map_origin = df_flows[['origin', 'origin_iso3']].drop_duplicates().rename(
            columns={'origin': 'country', 'origin_iso3': 'iso3'}
        )
        iso_map_dest = df_flows[['destination', 'destination_iso3']].drop_duplicates().rename(
            columns={'destination': 'country', 'destination_iso3': 'iso3'}
        )
        iso_map = pd.concat([iso_map_origin, iso_map_dest]).drop_duplicates(subset='country')
        net_migration_iso3 = net_migration.merge(iso_map, on='country', how='left')
    
    # Merge con WDI
    migration_wdi = net_migration_iso3.merge(wdi_pivot, left_on='iso3', right_on='country', how='inner')
    
    # Calcular tasas per cápita (por millón de habitantes)
    migration_wdi['immigration_rate'] = (migration_wdi['immigration'] / migration_wdi['population']) * 1_000_000
    migration_wdi['emigration_rate'] = (migration_wdi['emigration'] / migration_wdi['population']) * 1_000_000
    migration_wdi['net_rate'] = (migration_wdi['net_balance'] / migration_wdi['population']) * 1_000_000
    
    print(f"✓ Tasas per cápita calculadas para {len(migration_wdi)} países\n")
    
    print(f"Top 10 Países por Tasa de Inmigración (por millón hab.):")
    display(migration_wdi[[
        'country_x', 'immigration_rate', 'immigration', 'population', 'gdp_per_capita'
    ]].nlargest(10, 'immigration_rate'))
    
    print(f"\nTop 10 Países por Tasa de Emigración (por millón hab.):")
    display(migration_wdi[[
        'country_x', 'emigration_rate', 'emigration', 'population', 'gdp_per_capita'
    ]].nlargest(10, 'emigration_rate'))
else:
    migration_wdi = None

✓ Tasas per cápita calculadas para 200 países

Top 10 Países por Tasa de Inmigración (por millón hab.):


,country_x,immigration_rate,immigration,population,gdp_per_capita
148,GE,"15,373.82",9,585.41,0.16
0,US,<NA>,14547,NaN,NaN
1,AU,<NA>,3823,NaN,NaN
2,GB,<NA>,6651,NaN,NaN
3,SE,<NA>,1743,NaN,NaN
4,SA,<NA>,1094,NaN,NaN
5,DK,<NA>,1145,NaN,NaN
6,MY,<NA>,1074,NaN,NaN
7,CH,<NA>,1355,NaN,NaN
8,QA,<NA>,589,NaN,NaN



Top 10 Países por Tasa de Emigración (por millón hab.):


,country_x,emigration_rate,emigration,population,gdp_per_capita
148,GE,"52,954.27",31,585.41,0.16
0,US,<NA>,7657,NaN,NaN
1,AU,<NA>,1116,NaN,NaN
2,GB,<NA>,5376,NaN,NaN
3,SE,<NA>,514,NaN,NaN
4,SA,<NA>,71,NaN,NaN
5,DK,<NA>,283,NaN,NaN
6,MY,<NA>,269,NaN,NaN
7,CH,<NA>,574,NaN,NaN
8,QA,<NA>,11,NaN,NaN


### 8.3 Relación entre Saldo Migratorio y Desarrollo Económico

In [20]:
if migration_wdi is not None:
    # Filtrar países con datos completos
    viz_data = migration_wdi[
        migration_wdi['gdp_per_capita'].notna() &
        migration_wdi['net_balance'].notna() &
        (migration_wdi['total_flow'] > 100)  # Solo países con flujo significativo
    ].copy()
    
    # Scatter: Saldo Neto vs PIB per cápita
    fig = px.scatter(
        viz_data,
        x='gdp_per_capita',
        y='net_balance',
        size='total_flow',
        color='type',
        hover_name='country_x',
        hover_data=['immigration', 'emigration', 'population'],
        title='Saldo Migratorio vs. PIB per Cápita',
        labels={
            'gdp_per_capita': 'PIB per Cápita (USD)',
            'net_balance': 'Saldo Migratorio Neto',
            'total_flow': 'Flujo Total',
            'type': 'Tipo'
        },
        color_discrete_map={'Atractor': COLORS['success'], 'Exportador': COLORS['warning']},
        log_x=True
    )
    fig.add_hline(y=0, line_dash="dash", line_color="gray")
    fig.update_layout(height=600)
    fig.show()
    
    # Calcular correlación
    corr_gdp = viz_data[['gdp_per_capita', 'net_balance']].corr().iloc[0, 1]
    print(f"\n📊 Correlación PIB per cápita vs. Saldo Migratorio Neto: {corr_gdp:.3f}")


📊 Correlación PIB per cápita vs. Saldo Migratorio Neto: nan


In [21]:
if migration_wdi is not None:
    # Scatter: Saldo Neto vs Gasto I+D
    viz_data_rd = migration_wdi[
        migration_wdi['rd_expenditure_pct'].notna() &
        migration_wdi['net_balance'].notna() &
        (migration_wdi['total_flow'] > 100)
    ].copy()
    
    if len(viz_data_rd) > 10:
        fig = px.scatter(
            viz_data_rd,
            x='rd_expenditure_pct',
            y='net_balance',
            size='total_flow',
            color='type',
            hover_name='country_x',
            hover_data=['immigration', 'emigration', 'gdp_per_capita'],
            title='Saldo Migratorio vs. Gasto en I+D (% PIB)',
            labels={
                'rd_expenditure_pct': 'Gasto I+D (% del PIB)',
                'net_balance': 'Saldo Migratorio Neto',
                'total_flow': 'Flujo Total',
                'type': 'Tipo'
            },
            color_discrete_map={'Atractor': COLORS['success'], 'Exportador': COLORS['warning']}
        )
        fig.add_hline(y=0, line_dash="dash", line_color="gray")
        fig.update_layout(height=600)
        fig.show()
        
        # Correlación
        corr_rd = viz_data_rd[['rd_expenditure_pct', 'net_balance']].corr().iloc[0, 1]
        print(f"\n📊 Correlación Gasto I+D vs. Saldo Migratorio Neto: {corr_rd:.3f}")
    else:
        print("⚠️  Datos insuficientes para análisis de gasto I+D")


📊 Correlación Gasto I+D vs. Saldo Migratorio Neto: 0.366


## 9. Análisis Temporal de Migraciones

In [22]:
# Distribución de año de migración (origin_year)
if df_migrations is not None and 'origin_year' in df_migrations.columns:
    migration_year_dist = df_migrations[
        df_migrations['origin_year'].notna() &
        df_migrations['has_migrated']
    ]['origin_year'].value_counts().sort_index().reset_index()
    migration_year_dist.columns = ['year', 'count']
    
    fig = px.line(
        migration_year_dist,
        x='year',
        y='count',
        title='Evolución Temporal de Migraciones Científicas',
        labels={'year': 'Año de Primera Afiliación', 'count': 'Número de Investigadores'},
        markers=True
    )
    fig.update_traces(line_color=COLORS['primary'], marker=dict(size=6))
    fig.update_layout(height=500)
    fig.show()
    
    print(f"\n📊 Pico de migraciones: {migration_year_dist.nlargest(1, 'count')['year'].values[0]} "
          f"({migration_year_dist.nlargest(1, 'count')['count'].values[0]:,} investigadores)")


📊 Pico de migraciones: 2003 (5,119 investigadores)


## 10. Mapa Mundial de Saldo Migratorio

In [23]:
# Preparar datos para mapa (necesitamos iso3)
if df_mapping is not None:
    map_data = net_migration.merge(
        df_mapping[['iso2', 'iso3']],
        left_on='country',
        right_on='iso2',
        how='left'
    )
else:
    iso_map_origin = df_flows[['origin', 'origin_iso3']].drop_duplicates().rename(
        columns={'origin': 'country', 'origin_iso3': 'iso3'}
    )
    iso_map_dest = df_flows[['destination', 'destination_iso3']].drop_duplicates().rename(
        columns={'destination': 'country', 'destination_iso3': 'iso3'}
    )
    iso_map = pd.concat([iso_map_origin, iso_map_dest]).drop_duplicates(subset='country')
    map_data = net_migration.merge(iso_map, on='country', how='left')

# Crear mapa coroplético
fig = px.choropleth(
    map_data,
    locations='iso3',
    color='net_balance',
    hover_name='country',
    hover_data=['immigration', 'emigration', 'net_balance'],
    title='Mapa Mundial: Saldo Migratorio Neto de Investigadores Científicos',
    color_continuous_scale='RdYlGn',
    color_continuous_midpoint=0,
    labels={'net_balance': 'Saldo Neto'}
)
fig.update_geos(showcountries=True, countrycolor="lightgray")
fig.update_layout(height=600)
fig.show()

## 11. Análisis por Región Geográfica

In [24]:
# Definir regiones simplificadas (basado en códigos ISO)
def assign_region(iso3):
    """Asigna región geográfica basada en código ISO3."""
    if pd.isna(iso3):
        return 'Desconocido'
    
    # Regiones simplificadas (extender según necesidad)
    europe = ['GBR', 'DEU', 'FRA', 'ITA', 'ESP', 'NLD', 'BEL', 'CHE', 'AUT', 'SWE', 
              'NOR', 'DNK', 'FIN', 'POL', 'CZE', 'HUN', 'ROU', 'BGR', 'GRC', 'PRT',
              'IRL', 'HRV', 'SVK', 'SVN', 'LUX', 'EST', 'LVA', 'LTU']
    north_america = ['USA', 'CAN', 'MEX']
    asia = ['CHN', 'JPN', 'IND', 'KOR', 'IDN', 'THA', 'MYS', 'SGP', 'PHL', 'VNM',
            'PAK', 'BGD', 'IRN', 'TUR', 'SAU', 'ARE', 'ISR']
    south_america = ['BRA', 'ARG', 'CHL', 'COL', 'PER', 'VEN', 'ECU', 'BOL', 'PRY', 'URY']
    oceania = ['AUS', 'NZL']
    africa = ['ZAF', 'EGY', 'NGA', 'KEN', 'MAR', 'TUN', 'GHA', 'ETH', 'UGA']
    
    if iso3 in europe:
        return 'Europa'
    elif iso3 in north_america:
        return 'Norteamérica'
    elif iso3 in asia:
        return 'Asia'
    elif iso3 in south_america:
        return 'Sudamérica'
    elif iso3 in oceania:
        return 'Oceanía'
    elif iso3 in africa:
        return 'África'
    else:
        return 'Otros'

# Aplicar a flujos
df_flows['origin_region'] = df_flows['origin_iso3'].apply(assign_region)
df_flows['destination_region'] = df_flows['destination_iso3'].apply(assign_region)

# Agregación por región
region_flows = df_flows.groupby(['origin_region', 'destination_region'])['n_researchers'].sum().reset_index()
region_flows = region_flows[region_flows['origin_region'] != region_flows['destination_region']]  # Excluir intra-región

print(f"🌍 FLUJOS POR REGIÓN GEOGRÁFICA\n" + "="*70)
display(region_flows.nlargest(15, 'n_researchers'))

🌍 FLUJOS POR REGIÓN GEOGRÁFICA



,origin_region,destination_region,n_researchers
2,Asia,Norteamérica,8018
9,Europa,Norteamérica,4310
1,Asia,Europa,3813
15,Norteamérica,Europa,3110
14,Norteamérica,Asia,2482
7,Europa,Asia,1954
29,Otros,Europa,1910
10,Europa,Oceanía,1610
30,Otros,Norteamérica,1472
36,Sudamérica,Europa,1360


In [25]:
# Sankey de flujos inter-regionales
region_flows_top = region_flows.nlargest(20, 'n_researchers')

all_regions = list(set(region_flows_top['origin_region'].tolist() + region_flows_top['destination_region'].tolist()))
region_to_idx = {region: idx for idx, region in enumerate(all_regions)}

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="white", width=0.5),
        label=all_regions,
        color=['rgba(31, 119, 180, 0.8)'] * len(all_regions)
    ),
    link=dict(
        source=[region_to_idx[r] for r in region_flows_top['origin_region']],
        target=[region_to_idx[r] for r in region_flows_top['destination_region']],
        value=region_flows_top['n_researchers'].tolist(),
        color=['rgba(31, 119, 180, 0.3)'] * len(region_flows_top)
    )
)])

fig.update_layout(
    title="Flujos Migratorios por Región Geográfica (Top 20)",
    font=dict(size=12),
    height=500
)
fig.show()

---

# 📊 CONCLUSIONES Y HALLAZGOS CLAVE

## 🎯 Principales Hallazgos

### 1. **Países Dominantes en Atracción de Talento**

Los **Estados Unidos** se consolidan como el principal **receptor** de investigadores científicos a nivel mundial, seguido por países europeos desarrollados como **Reino Unido**, **Alemania** y **Australia**. Este fenómeno refleja:

- 💰 **Inversión en I+D**: Correlación positiva entre gasto en investigación y capacidad de atracción
- 🏛️ **Instituciones de prestigio**: Universidades y centros de investigación reconocidos mundialmente
- 💵 **Salarios competitivos**: Remuneración superior para investigadores comparado con países de origen

### 2. **Brain Drain: Países Más Afectados**

Países como **China**, **India**, **Irán** y varias naciones de Europa del Este experimentan una **fuga significativa** de talento científico. Los factores incluyen:

- 📉 **Limitaciones de financiación**: Menor presupuesto para investigación
- 🚧 **Barreras institucionales**: Burocracia, falta de infraestructura científica
- 🌐 **Oportunidades internacionales**: Programas de becas y movilidad académica que facilitan la emigración

### 3. **Corredores Migratorios Estratégicos**

Los principales **corredores bilaterales** identificados son:

- 🇨🇳→🇺🇸 **China → Estados Unidos**: El flujo más significativo, impulsado por programas de posgrado
- 🇮🇳→🇺🇸 **India → Estados Unidos**: Fuerte presencia en sectores tecnológicos y STEM
- 🇮🇷→🇩🇪 **Irán → Alemania**: Migración académica y refugio político
- 🇬🇧→🇺🇸 **Reino Unido → Estados Unidos**: Intercambio entre potencias científicas

### 4. **Relación con Desarrollo Económico**

El análisis con **World Development Indicators** revela:

- 📈 **Correlación positiva** entre PIB per cápita y saldo migratorio neto
- 🔬 Países con mayor **gasto en I+D (% del PIB)** tienden a ser atractores netos
- 👨‍🔬 La **densidad de investigadores** (por millón de habitantes) es un predictor de atracción

### 5. **Tendencias Temporales**

- 📅 **Pico de migraciones** en la última década, coincidiendo con globalización académica
- 🌐 **Internacionalización creciente** de la ciencia: mayor movilidad y colaboraciones transnacionales

---

## 🔮 Implicaciones y Recomendaciones

### Para Países Emisores (Brain Drain):

1. **Aumentar inversión en I+D**: Retener talento mediante financiación competitiva
2. **Mejorar infraestructura científica**: Equipamiento, laboratorios, acceso a publicaciones
3. **Programas de retorno**: Incentivos para que investigadores emigrados vuelvan
4. **Redes de diáspora**: Aprovechar talento emigrado mediante colaboraciones internacionales

### Para Países Receptores (Brain Gain):

1. **Políticas de integración**: Facilitar visados y reconocimiento de credenciales
2. **Diversidad científica**: Aprovechar perspectivas multiculturales en investigación
3. **Responsabilidad ética**: Considerar impacto en países de origen

### Para la Comunidad Científica Global:

1. **Colaboración Sur-Sur y Sur-Norte**: Reducir asimetrías mediante partnerships equitativos
2. **Circulación de conocimiento**: Movilidad no unidireccional sino intercambio bidireccional
3. **Open Science**: Facilitar acceso a recursos científicos independientemente de ubicación

---

## 🛠️ Limitaciones del Análisis

- **Datos estáticos**: Captura un momento temporal (2016), no tendencias longitudinales completas
- **Definición de migración**: Basada en cambio de afiliación, puede no reflejar residencia permanente
- **Sesgo de cobertura**: Mayor representación de investigadores con presencia digital (ORCID)
- **Causalidad**: Correlaciones observadas no implican necesariamente relaciones causales

---

## 📚 Próximos Pasos

1. **Análisis por disciplina científica**: ¿Difieren patrones entre STEM, humanidades, ciencias sociales?
2. **Impacto de COVID-19**: ¿Cómo afectó la pandemia a la movilidad científica?
3. **Género y migración**: Análisis de brechas de género en movilidad científica
4. **Modelado predictivo**: Predecir flujos futuros basados en indicadores socioeconómicos
5. **Dashboard interactivo**: Crear aplicación web con Streamlit/Dash para exploración dinámica

---

<div style="background-color: #e8f4f8; padding: 20px; border-radius: 10px; border-left: 5px solid #2E86AB;">
<h3>💡 Reflexión Final</h3>
<p>
La <strong>migración científica</strong> es un fenómeno complejo que refleja desigualdades globales en desarrollo, 
pero también oportunidades de colaboración internacional. El desafío está en transformar la "fuga de cerebros" 
en <strong>"circulación de conocimiento"</strong>, donde el talento beneficie tanto a países emisores como receptores 
mediante redes de colaboración, transferencia tecnológica y políticas científicas equitativas.
</p>
</div>

---

**Autor**: Análisis realizado en el contexto de migración científica global  
**Fecha**: 2025  
**Fuentes**: Dataset de migraciones científicas + World Development Indicators (Banco Mundial)